In [ ]:
!pip install optuna


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import gc
import random
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,roc_auc_score
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import log_loss
import lightgbm as lgbm


In [4]:
from sklearn.impute   import SimpleImputer

train1 = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/test.csv')
columns = [col for col in train1.columns.to_list() if col not in ['id','song_popularity']]
data=train1[columns]
target=train1['song_popularity']

median_impute  = SimpleImputer(strategy='median')
imputed_data = median_impute.fit_transform(data)
imputed_data = pd.DataFrame(imputed_data, columns = data.columns)
train=imputed_data

In [5]:
NUM_CLASSES=target.nunique()

In [6]:
train

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,0.135945,-5.619088,0.0,0.082570,158.386236,4.0,0.734642
1,186660.0,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1.0,0.127358,102.752988,3.0,0.711531
2,193213.0,0.140532,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0.0,0.052282,178.685791,3.0,0.425536
3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0.0,0.035618,128.715630,3.0,0.453597
4,165969.0,0.493017,0.608234,0.740982,0.002033,10.0,0.094891,-2.684095,0.0,0.050746,121.928157,4.0,0.741311
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,237799.0,0.140532,0.748942,0.511234,0.002913,0.0,0.173803,-8.853673,0.0,0.078060,168.262924,3.0,0.178159
39996,191119.0,0.067488,0.672830,0.889685,0.001225,5.0,0.122924,-7.798993,0.0,0.188607,110.684544,3.0,0.790626
39997,160879.0,0.877431,0.409065,0.292671,0.001974,10.0,0.110664,-15.378585,0.0,0.031294,99.556074,3.0,0.177947
39998,193918.0,0.140532,0.365738,0.704453,0.000339,1.0,0.356308,-4.661977,1.0,0.054096,139.857384,3.0,0.772978


In [7]:
params={'bagging_fraction': 0.7,
 'bagging_freq': 1,
 'feature_fraction': 0.5,
 'lambda_l1': 0,
 'lambda_l2': 30,
 'learning_rate': 0.12175906487922063,
 'max_depth': 8,
 'min_data_in_leaf': 3100,
 'min_gain_to_split': 1.543021792185368,
 'n_estimators': 10000,
 'num_leaves': 2340}

In [11]:
from tqdm import tqdm
np.random.seed(64)
random.seed(64)


target = train1['song_popularity'].values
FOLDS=10
kf = StratifiedKFold(n_splits = FOLDS, shuffle=True, random_state=64)    
        
oof = np.zeros((train.shape[0],))
predictions = 0
auc_value = []
for f, (train_index, test_index) in tqdm(enumerate(kf.split(X=train, y=target))):
        X_train, X_val = train.iloc[train_index][columns], train.iloc[test_index][columns]
        y_train, y_val = target[train_index], target[test_index]
        model = lgbm.LGBMClassifier(**params)
        model.fit(X_train[columns],
                  y_train,
                  eval_set=[(X_val[columns], y_val)],
                  early_stopping_rounds=300,
                  verbose=100
                  )
  
        
        oof_temp_pred = model.predict_proba(X_val[columns])[:,1]
        test_temp_pred = model.predict_proba(test[columns])[:,1]
        oof[test_index] = oof_temp_pred
        predictions += test_temp_pred/FOLDS
        auc = roc_auc_score(y_val, oof_temp_pred)
        auc_value.append(auc)
        print(f'FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_value)}')

0it [00:00, ?it/s]

Training until validation scores don't improve for 300 rounds.
[100]	valid_0's binary_logloss: 0.649066
[200]	valid_0's binary_logloss: 0.648985
[300]	valid_0's binary_logloss: 0.649338
[400]	valid_0's binary_logloss: 0.649338


1it [00:00,  1.53it/s]

Early stopping, best iteration is:
[153]	valid_0's binary_logloss: 0.648558
FOLD: 0 AUC: 0.5734028113833958 Mean AUC: 0.5734028113833958
Training until validation scores don't improve for 300 rounds.
[100]	valid_0's binary_logloss: 0.650178
[200]	valid_0's binary_logloss: 0.649796
[300]	valid_0's binary_logloss: 0.649909
[400]	valid_0's binary_logloss: 0.649918
[500]	valid_0's binary_logloss: 0.649549
[600]	valid_0's binary_logloss: 0.649703
[700]	valid_0's binary_logloss: 0.649653
[800]	valid_0's binary_logloss: 0.64979
Early stopping, best iteration is:
[562]	valid_0's binary_logloss: 0.649428


2it [00:01,  1.04s/it]

...

FOLD: 7 AUC: 0.5726831750595482 Mean AUC: 0.5715578285863139
Training until validation scores don't improve for 300 rounds.
[100]	valid_0's binary_logloss: 0.65011
[200]	valid_0's binary_logloss: 0.649822
[300]	valid_0's binary_logloss: 0.649822


9it [00:06,  1.53it/s]

[400]	valid_0's binary_logloss: 0.649822
Early stopping, best iteration is:
[155]	valid_0's binary_logloss: 0.649735
FOLD: 8 AUC: 0.5657041267744418 Mean AUC: 0.5709074172738836
Training until validation scores don't improve for 300 rounds.
[100]	valid_0's binary_logloss: 0.645247
[200]	valid_0's binary_logloss: 0.645162
[300]	valid_0's binary_logloss: 0.645291
[400]	valid_0's binary_logloss: 0.645241
[500]	valid_0's binary_logloss: 0.645069
[600]	valid_0's binary_logloss: 0.645005
[700]	valid_0's binary_logloss: 0.645104
[800]	valid_0's binary_logloss: 0.645104
Early stopping, best iteration is:
[570]	valid_0's binary_logloss: 0.644838


10it [00:07,  1.29it/s]

FOLD: 9 AUC: 0.5840569785626172 Mean AUC: 0.572222373402757


In [12]:
submission_df =pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/sample_submission.csv')
submission_df['song_popularity'] = predictions
submission_df.to_csv('submission_lgbm_best_params1_200_trials_imputed_median_300_rounds.csv', index=False)